## Task

Build a model with an F1 measure greater than 0.59.

## Data Preparations

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv')
data.head()

First, we delete the columns that are unnecessary for our task.

In [ ]:
del data['RowNumber'], data['CustomerId'], data['Surname']

**Gender** and **Geography** are categorical variables. We will generate dummy columns for them.

In [ ]:
gender_dummies = pd.get_dummies(data['Gender'])
country_dummies = pd.get_dummies(data['Geography'])
data = pd.concat([data, gender_dummies, country_dummies], axis=1)

Let's clear the table of already unnecessary columns.

In [ ]:
del data['Gender'], data['Geography']

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isna().sum()

### Conclusion

Data has been studied and prepared for research. Dummy columns are created for categorical variables, you can proceed to the next step.

## Task Research

Let's try the decision tree first.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


y = data['Exited']
x = data.drop(['Exited'], axis = 1)

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

data_train, data_valid, data_test = train_validate_test_split(data, seed=12345)

x_train = data_train.drop(['Exited'], axis = 1)
y_train = data_train['Exited']

x_valid = data_valid.drop(['Exited'], axis = 1)
y_valid = data_valid['Exited']

x_test = data_test.drop(['Exited'], axis = 1)
y_test = data_test['Exited']

In [ ]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
result = model.predict(x_valid)
print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

We got an unsatisfactory F1 metric and a good AUC score. We will study columns and other metrics for this dataset.

In [ ]:
data['Exited'].value_counts()

A big discrepancy between those who left the program and those remaining, you can try to fix this.

In [ ]:
7963/2037

The discrepancy is almost 4 times !!!

## Unbalance

First, try to play around with the arguments and model type.

In [ ]:
from sklearn.linear_model import LogisticRegression

linear_model = LogisticRegression(random_state=12345)
linear_model.fit(x_train, y_train)
result = linear_model.predict(x_valid)

print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = linear_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

This model didn’t say anything at all, even though the AUC score is pretty good

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier(random_state=12345)
forest_model.fit(x_train, y_train)
result = forest_model.predict(x_valid)

print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = forest_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

Here it’s already better than a decisive tree, while we stop the choice on this model, try to twist its arguments. Also note that the AUC score has grown significantly, we will choose this model for improvement.

In [ ]:
best_estim = None
best_depth = None
best_f1_score = None
for estimators in range (5, 121, 10):
    for depth in range (5, 26, 5):
        forest_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=estimators)
        forest_model.fit(x_train, y_train)
        result = forest_model.predict(x_valid)
        score = f1_score(result, y_valid)
        if best_f1_score is None:
            best_f1_score = score
            best_estim = estimators
            best_depth = depth
        elif best_f1_score < score:
            best_f1_score = score
            best_estim = estimators
            best_depth = depth
        print("{0} estim, {1} depth, {2} score".format(estimators, depth, score))

In [ ]:
print("Estimators - {0}, Depth - {1}, F1 Score - {2}".format(best_estim, best_depth, best_f1_score))

Already reached 0.547, not bad! For the task you need 0.59, let's try to fix it with the balance of classes.

In [ ]:
forest_model = RandomForestClassifier(random_state=12345, class_weight='balanced', n_estimators=best_estim, max_depth=best_depth)
forest_model.fit(x_train, y_train)
result = forest_model.predict(x_valid)
print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = forest_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

It became a little better, based on the last step, we will try to do upsampling of some classes.

In [ ]:
def upsample(features, target, repeat):
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    return features_upsampled, target_upsampled

x_upsampled, y_upsampled = upsample(x_train, y_train, 4)

In [ ]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=best_estim, max_depth=best_depth,class_weight='balanced')
forest_model.fit(x_upsampled, y_upsampled)
result = forest_model.predict(x_valid)
print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = forest_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

We got a slightly better metric, try without balance.

In [ ]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=best_estim, max_depth=best_depth)
forest_model.fit(x_upsampled, y_upsampled)
result = forest_model.predict(x_valid)
print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = forest_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

Without balance, it turned out better. Let's try to do Downsampling.

In [ ]:
from sklearn.utils import shuffle

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

x_downsampled, y_downsampled = downsample(x_train, y_train, 0.25)

In [ ]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=best_estim, max_depth=best_depth, class_weight='balanced')
forest_model.fit(x_downsampled, y_downsampled)
result = forest_model.predict(x_valid)
print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = forest_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

In [ ]:
forest_model = RandomForestClassifier(random_state=12345, n_estimators=best_estim, max_depth=best_depth)
forest_model.fit(x_downsampled, y_downsampled)
result = forest_model.predict(x_valid)
print("F1 Score {0}".format(f1_score(y_valid, result)))
probs = forest_model.predict_proba(x_valid)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_valid, probs)))
fpr, tpr, thresholds = roc_curve(y_valid, probs)
plot_roc_curve(fpr, tpr)

It turned out worse, which does not reach our goal, we will stop at Upsampling without balance.

### Conclusion

In [ ]:
x_upsampled, y_upsampled = upsample(x_train, y_train, 4)

forest_model = RandomForestClassifier(random_state=12345, n_estimators=best_estim, max_depth=best_depth)
forest_model.fit(x_upsampled, y_upsampled)
result = forest_model.predict(x_test)
f1_score(result, y_test)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

probs = forest_model.predict_proba(x_test)
probs = probs[:, 1]

print("AUC Score {0}".format(roc_auc_score(y_test, probs)))
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

It would be nice if there was a unit, but it will do! The result is pretty good, I think we are satisfied. The result is much higher than 0.5, which means that our classifier works well and we are on the right track.

### Conclusion

For the entire project, a model was constructed that satisfies the conditions of the assignment, other metrics were also used to validate the result. We looked at several variants of models for the problem, opted for the forest as the most accurate model and brought the F1 metric slightly above 0.59 using the Upsampling technique.